# Imports

In [2]:
import pandas as pd
import numpy as np

import torch
import requests
from bs4 import BeautifulSoup
import re

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Set up model

In [3]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# Encode and Analyze Sentiment

In [4]:
tokens = tokenizer.encode("I hated this, absolutely the worst", return_tensors='pt')

In [5]:
tokenizer.decode(tokens[0])

'[CLS] i hated this, absolutely the worst [SEP]'

In [6]:
results = model(tokens)

In [7]:
results

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [8]:
int(torch.argmax(results.logits)) +1

1

# Grab some reviews

In [9]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
result = soup.find_all('p', {'class':regex})
reviews = [review.text for review in result]

In [12]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [13]:
df

,review
0,The food was delicious. The ricotta pancakes w...
1,Very cute coffee shop and restaurant. They hav...
2,Six of us met here for breakfast before our wa...
3,Great place with delicious food and friendly s...
4,Great food amazing coffee and tea. Short walk ...
5,"Great service, lovely location, and really ama..."
6,Ricotta hot cakes! These were so yummy. I ate ...
7,It was ok. Had coffee with my friends. I'm new...
8,We came for brunch twice in our week-long visi...
9,I came to Social brew cafe for brunch while ex...


In [14]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) +1

In [15]:
df["score"] = df["review"].apply(lambda review: sentiment_score(review))

In [16]:
df

,review,score
0,The food was delicious. The ricotta pancakes w...,4
1,Very cute coffee shop and restaurant. They hav...,5
2,Six of us met here for breakfast before our wa...,4
3,Great place with delicious food and friendly s...,5
4,Great food amazing coffee and tea. Short walk ...,5
5,"Great service, lovely location, and really ama...",5
6,Ricotta hot cakes! These were so yummy. I ate ...,5
7,It was ok. Had coffee with my friends. I'm new...,3
8,We came for brunch twice in our week-long visi...,5
9,I came to Social brew cafe for brunch while ex...,5
